In [1]:
import numpy as np
import pandas as pd

from pycaret.regression import *
from sklearn.model_selection import train_test_split

# preparaing datasets

In [27]:
df = pd.read_csv('inputs/USDJPY_M15.csv', sep='\t', names=['date', 'time', 'open', 'high', 'low', 'close', 'vol1', 'vol2', 'spread'], header=0)
df['datetime'] = df['date'] + ' ' + df['time']
df = df.drop(['date', 'time', 'vol1', 'vol2', 'spread'], axis=1)
df['time'] = pd.to_datetime(df['datetime'])
df['month'] = [i.month for i in df['time']]
df['year'] = [i.year for i in df['time']]
df['hour'] = [i.hour for i in df['time']]
df['minute'] = [i.minute for i in df['time']]
df['dayofweek'] = [i.dayofweek for i in df['time']]
# for i in range(1, 13):
#     df['diff%s'%i] = df['high'].shift(i)

df = df.drop(['datetime'], axis=1)
df

,open,high,low,close,time,month,year,hour,minute,dayofweek
0,107.547,107.567,107.455,107.518,2020-04-01 00:00:00,4,2020,0,0,2
1,107.520,107.545,107.469,107.540,2020-04-01 00:15:00,4,2020,0,15,2
2,107.540,107.544,107.476,107.492,2020-04-01 00:30:00,4,2020,0,30,2
3,107.492,107.519,107.465,107.481,2020-04-01 00:45:00,4,2020,0,45,2
4,107.481,107.549,107.481,107.547,2020-04-01 01:00:00,4,2020,1,0,2
...,...,...,...,...,...,...,...,...,...,...
26874,108.814,108.846,108.808,108.845,2021-04-30 06:15:00,4,2021,6,15,4
26875,108.845,108.859,108.831,108.846,2021-04-30 06:30:00,4,2021,6,30,4
26876,108.846,108.852,108.827,108.829,2021-04-30 06:45:00,4,2021,6,45,4
26877,108.829,108.851,108.822,108.827,2021-04-30 07:00:00,4,2021,7,0,4


In [28]:
df['y'] = df['high'].shift(-1)
df = df[:-1]
df

,open,high,low,close,time,month,year,hour,minute,dayofweek,y
0,107.547,107.567,107.455,107.518,2020-04-01 00:00:00,4,2020,0,0,2,107.545
1,107.520,107.545,107.469,107.540,2020-04-01 00:15:00,4,2020,0,15,2,107.544
2,107.540,107.544,107.476,107.492,2020-04-01 00:30:00,4,2020,0,30,2,107.519
3,107.492,107.519,107.465,107.481,2020-04-01 00:45:00,4,2020,0,45,2,107.549
4,107.481,107.549,107.481,107.547,2020-04-01 01:00:00,4,2020,1,0,2,107.549
...,...,...,...,...,...,...,...,...,...,...,...
26873,108.779,108.819,108.769,108.814,2021-04-30 06:00:00,4,2021,6,0,4,108.846
26874,108.814,108.846,108.808,108.845,2021-04-30 06:15:00,4,2021,6,15,4,108.859
26875,108.845,108.859,108.831,108.846,2021-04-30 06:30:00,4,2021,6,30,4,108.852
26876,108.846,108.852,108.827,108.829,2021-04-30 06:45:00,4,2021,6,45,4,108.851


# feature engineering

In [29]:
# df['sma5'] = df['high'].rolling(5).mean()
# df['sma15'] = df['high'].rolling(15).mean()
df = df[20:]
df.head(20)

,open,high,low,close,time,month,year,hour,minute,dayofweek,y
20,107.710,107.936,107.668,107.861,2020-04-01 05:00:00,4,2020,5,0,2,107.899
21,107.861,107.899,107.746,107.787,2020-04-01 05:15:00,4,2020,5,15,2,107.835
22,107.791,107.835,107.733,107.778,2020-04-01 05:30:00,4,2020,5,30,2,107.785
23,107.778,107.785,107.691,107.692,2020-04-01 05:45:00,4,2020,5,45,2,107.707
24,107.691,107.707,107.629,107.681,2020-04-01 06:00:00,4,2020,6,0,2,107.739
25,107.681,107.739,107.641,107.703,2020-04-01 06:15:00,4,2020,6,15,2,107.840
26,107.701,107.840,107.659,107.790,2020-04-01 06:30:00,4,2020,6,30,2,107.821
27,107.791,107.821,107.756,107.792,2020-04-01 06:45:00,4,2020,6,45,2,107.827
28,107.793,107.827,107.757,107.779,2020-04-01 07:00:00,4,2020,7,0,2,107.817
29,107.779,107.817,107.730,107.732,2020-04-01 07:15:00,4,2020,7,15,2,107.772


In [30]:
df.isnull().sum()

open         0
high         0
low          0
close        0
time         0
month        0
year         0
hour         0
minute       0
dayofweek    0
y            0
dtype: int64

In [33]:
train, test = train_test_split(df)
train.shape, test.shape

((20143, 11), (6715, 11))

In [34]:
s = setup(data=train, test_data=test, target='y', silent=True, session_id=123)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:36:35
Status,. . . . . . . . . . . . . . . . . .,Preparing Data for Modeling


ValueError: Setting a random_state has no effect since shuffle is False. You should leave random_state to its default (None), or set shuffle=True.

In [ ]:
best = compare_models(sort = 'MAE')

In [ ]:
prediction_holdout = predict_model(best);

In [ ]:
# generate predictions on the original dataset
predictions = predict_model(best, data=df)

In [ ]:
predictions

In [ ]:
tuned_model = tune_model(best)

In [ ]:
bagged_model = ensemble_model(tuned_model)

In [ ]:
boosted_model = ensemble_model(tuned_model, method = 'Boosting')

In [ ]:
top3 = compare_models(n_select = 3)

In [ ]:
blender_top3 = blend_models(top3)